# A. Model Purpose

**Rekomendasi produk skincare berdasarkan permasalahan kulit user.**

Dalam sistem ini, rekomendasi dihasilkan dengan cara mengukur kesamaan makna semantik antara keluhan pengguna dan deskripsi produk skincare.Pendekatan ini menggunakan Word2Vec untuk memahami makna dan konteks di balik kata-kata tersebut.

Model Word2Vec memetakan setiap kata ke dalam bentuk vektor numerik di ruang dimensi tinggi.
Dengan begitu, kata-kata yang memiliki makna serupa (misalnya kering, kusam, kasar) akan memiliki posisi vektor yang berdekatan.
Kesamaan antar teks diukur menggunakan cosine similarity antar rata-rata vektor deskripsi produk dan vektor keluhan pengguna.

Pendekatan ini termasuk ke dalam metode Content-Based Filtering, artinya:

>Sistem merekomendasikan produk yang secara konten paling mirip dengan kebutuhan atau keluhan pengguna berdasarkan makna dari deskripsi produk, bukan perilaku pembelian pengguna lain.

Karena model ini tidak memiliki label target, maka termasuk dalam kategori unsupervised learning, dengan karakteristik:

- Tidak ada data label (target class).
- Sistem hanya mengukur kedekatan makna antar teks menggunakan representasi vektor Word2Vec.
- Model tidak belajar dari contoh eksplisit, tetapi dari pola hubungan kata di dalam deskripsi produk.

# B. Notebook Data Processing Content
---

1. Import packages
2. Load data
3. Feature Engineering


---
## B-1. Import Package


In [ ]:
!pip install gensim==4.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pyten

In [ ]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install nltk contractions
# # Install it after sekali jeee

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.1/345.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00


In [ ]:
# Setup
import os
import re
import string
import numpy as np
import pandas as pd
import pickle
import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML

# NLP & Text Processing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Word2Vec (ganti TF-IDF)
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


---
## B-2. Load data


In [ ]:
df = pd.read_csv('skincare_clean.csv')

In [ ]:
display(HTML('<center><b><h1>Top 5 : dataset</h1></b></center><br>'))
display(df.head(5))

,product_type,brand,product,rating,review_count,url,image_url,price,description,unique_id
0,cleanser,Cetaphil,Gentle Skin Cleanser,4.2,15298,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,112000.0,Cetaphil Gentle Skin Cleanser mengandung formu...,Cetaphil - Gentle Skin Cleanser
1,cleanser,Senka,Perfect Whip Facial Foam,4.2,6374,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,200000.0,Shiseido Perfect Whip adalah pembersih wajah u...,Senka - Perfect Whip Facial Foam
2,cleanser,Acnes,Creamy Wash,3.7,5510,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,29000.0,Acnes Creamy Wash adalah sabun pembersih wajah...,Acnes - Creamy Wash
3,cleanser,Hada Labo,Tamagohada Mild Peeling Face Wash,4.1,5147,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,35000.0,Hada Labo Tamagohada Mild Peeling Face Wash ad...,Hada Labo - Tamagohada Mild Peeling Face Wash
4,cleanser,Hada Labo,Gokujyun Ultimate Moisturizing Face Wash,4.3,4468,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,30000.0,Hada Labo Gokujyun Ultimate Moisturizing Face ...,Hada Labo - Gokujyun Ultimate Moisturizing Fac...


In [ ]:
display(HTML('<center><b><h1>Bottom 5 : dataset</h1></b></center><br>'))
display(df.tail(5))

,product_type,brand,product,rating,review_count,url,image_url,price,description,unique_id
783,toner,Dermies,Hello Glow Radiance Essence Toner,4.28,96,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,75000.0,Hello Glow Radiance Essence Toner merupakan pe...,Dermies - Hello Glow Radiance Essence Toner
784,toner,A.Stop,Clear Face & Body Toner,3.70,94,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,255000.0,A.Stop Clear Face & Body Toner is the 1 stop s...,A.Stop - Clear Face & Body Toner
785,toner,Hada Labo,Gokujyun Super Hyaluronic Acid Hydrating Lotio...,4.30,94,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,200000.0,Contains 3 types of Hyaluronic Acid that work ...,Hada Labo - Gokujyun Super Hyaluronic Acid Hyd...
786,toner,Innisfree,Aloe Revital Skin Mist,4.00,92,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,125000.0,The mist-type toner containing Jeju chemical-f...,Innisfree - Aloe Revital Skin Mist
787,toner,By Wishtrend,Mandelic Acid 5% Skin Prep Water,4.20,92,https://reviews.femaledaily.com/products/clean...,https://image.femaledaily.com/dyn/210/images/p...,300000.0,By Wishtrend Mandelic Acid 5% Skin Prep Water ...,By Wishtrend - Mandelic Acid 5% Skin Prep Water


---
### B-2.1. Checking Data Structure and Missing Value

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 788 entries, 0 to 787
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_type         788 non-null    object 
 1   brand                788 non-null    object 
 2   product              788 non-null    object 
 3   rating               788 non-null    float64
 4   review_count         788 non-null    int64  
 5   url                  788 non-null    object 
 6   image_url            788 non-null    object 
 7   price                788 non-null    float64
 8   description          788 non-null    object 
 9   unique_id            788 non-null    object 
 10  cleaned_description  788 non-null    object 
 11  tokens               788 non-null    object 
 12  w2v_vector           788 non-null    object 
dtypes: float64(2), int64(1), object(10)
memory usage: 80.2+ KB


Dataset ini berisi 788 produk skincare dengan 13 kolom informasi.
Setiap data mencakup detail seperti jenis produk, merek, nama produk, harga, rating, jumlah ulasan, tautan gambar, serta deskripsi produk.
Ukuran dataset relatif kecil dengan kapasitas sekitar 80 KB, sehingga mudah untuk dianalisis lebih lanjut.

In [ ]:
print('\n Checking Missing Value')
print('-'*50)
print(df.isna().sum())


 Checking Missing Value
--------------------------------------------------
product_type           0
brand                  0
product                0
rating                 0
review_count           0
url                    0
image_url              0
price                  0
description            0
unique_id              0
cleaned_description    0
tokens                 0
w2v_vector             0
dtype: int64


Hasil pemeriksaan menunjukkan bahwa seluruh kolom dalam dataset tidak memiliki nilai kosong (missing value).
Artinya, setiap atribut seperti jenis produk, merek, harga, rating, dan deskripsi terisi lengkap pada semua 788 data produk skincare.
Kondisi ini memastikan data sudah bersih dan siap digunakan untuk tahap analisis berikutnya.

In [ ]:
print('\n === Unique Value: kolom brand ===')
print('-'*80)
display(df['brand'].unique()[:20])
print(f'Jumlah Unique Value dari kolom : ', (df['brand'].nunique()))

print('\n === Unique Value: kolom product ===')
print('-'*80)
display(df['product'].unique()[:10])
print(f'Jumlah Unique Value dari kolom : ', (df['product'].nunique()))

print('\n === Unique Value: kolom product_type ===')
print('-'*80)
display(df['product_type'].unique()[:10])
print(f'Jumlah Unique Value dari kolom : ', (df['product_type'].nunique()))

print('\n === Unique Value: kolom description ===')
print('-'*80)
display(df['description'].unique()[:10])
print(f'Jumlah Unique Value dari kolom : ', (df['description'].nunique()))



 === Unique Value: kolom brand ===
--------------------------------------------------------------------------------


array(['Cetaphil', 'Senka', 'Acnes', 'Hada Labo', 'Cosrx',
       'The Body Shop', "Pond's", 'NIVEA', 'Wardah', 'Glow & Lovely',
       'NPURE', 'Kojic Plankton', 'Clean & Clear', 'Kleveru Organics',
       'Sariayu', 'SK-II', 'Innisfree', 'White Story', 'Azarine Cosmetic',
       'Emina'], dtype=object)

Jumlah Unique Value dari kolom :  208

 === Unique Value: kolom product ===
--------------------------------------------------------------------------------


array(['Gentle Skin Cleanser', 'Perfect Whip Facial Foam', 'Creamy Wash',
       'Tamagohada Mild Peeling Face Wash',
       'Gokujyun Ultimate Moisturizing Face Wash',
       'Low pH Good Morning Gel Cleanser',
       'Tea Tree Skin Clearing Facial Wash',
       'Bright Miracle Ultimate Clarity Facial Foam',
       'Sparkling Bright Facial Foam', 'Lightening Micellar Gentle Wash'],
      dtype=object)

Jumlah Unique Value dari kolom :  763

 === Unique Value: kolom product_type ===
--------------------------------------------------------------------------------


array(['cleanser', 'serum/essence', 'sun protection', 'moisturizer',
       'toner'], dtype=object)

Jumlah Unique Value dari kolom :  5

 === Unique Value: kolom description ===
--------------------------------------------------------------------------------


array(['Cetaphil Gentle Skin Cleanser mengandung formula gel yang telah terbukti secara klinis memberikan hidrasi yang sangat baik untuk melindungi dari kekeringan, sambil membiarkan kulit terhidrasi setelah dibersihkan. Diformulasikan untuk menghilangkan kotoran, riasan dan debu dengan lembut namun efektif, ditambah campuran niacinamide (vitamin B3), panthenol (pro-vitamin B5) dan gliserin yang menghidrasi untuk menjaga lapisan kelembapan alami kulit. Melindungi dari 5 tanda sensitivitas kulit termasuk lapisan kelembapan terganggu, tidak nyaman, dehidrasi, kaku, kasar.How To Use:Aplikasikan pada bagian wajah dan tubuh. Dapat digunakan dengan atau tanpa air.Suitable For:Semua jenis kulitIngredients:Aqua, Glycerin, Cetearyl Alcohol, Panthenol, Niacinamide, Pantolactone, Xanthan Gum, Sodium Cocoyl Isethionate, Sodium Benzoate, Citric Acid. FIL.1747.V00',
       'Shiseido Perfect Whip adalah pembersih wajah untuk menghilangkan kotoran, minyak berlebih dan sisa make up, dan pada saat bersa

Jumlah Unique Value dari kolom :  787


---
## B-3. Feature Engineering
---

---
### B-3.2. Text Preprocessing

In [ ]:
%%time
# Inisialisasi tools
factory = StemmerFactory()
stemmer = factory.create_stemmer()

indo_stopwords = set(stopwords.words('indonesian'))

# Fungsi preprocess
def preprocess_text(text):
    if pd.isna(text):
        return text
    # bersihkan garis baru dan spasi berlebih
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    # tambah spasi sebelum teks yang nempel
    keywords = ['ingredients', 'how to use', 'suitable']
    pattern = r'(?i)([a-zA-Z0-9])(' + '|'.join([k.replace(' ', '') for k in keywords]) + r')'
    text = re.sub(pattern, lambda m: f'{m.group(1)} {m.group(2)}', text)

    # lowercase
    text = text.lower()

    # hapus angka dan satuan
    text = re.sub(r'\bx\b', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'\b\d+[a-zA-Z]+\b', '', text)

    # hapus tanda baca
    text = text.translate(str.maketrans('', '', string.punctuation))

    # hilangkan kata satuan umum
    satuan_umum = {'ml', 'gram', 'menit', 'pump', 'pcs'}
    text = ' '.join([word for word in text.split() if word not in satuan_umum])

    # hapus kata kunci tidak relevan
    for k in keywords:
        text = re.sub(k + r'\s*:?', k, text)

    # stopwords
    text = ' '.join([word for word in text.split() if word not in indo_stopwords])

    # bersihkan spasi berlebih
    text = re.sub(r'\s+', ' ', text).strip()

    return text

df['cleaned_description'] = df['description'].apply(preprocess_text)

CPU times: user 224 ms, sys: 1.15 ms, total: 226 ms
Wall time: 227 ms


Proses text preprocessing dilakukan untuk membersihkan teks deskripsi produk dari karakter yang tidak relevan seperti angka, tanda baca, satuan (ml, gram, pcs), dan kata umum yang tidak memberi makna penting.
Seluruh teks diubah menjadi huruf kecil dan dihapus stopwords bahasa Indonesia agar lebih konsisten.

Pada tahap ini stemming tidak diterapkan, karena banyak kata dalam deskripsi skincare merupakan istilah teknis atau serapan bahasa Inggris seperti hydrating, brightening, atau moisturizing.
Jika dilakukan stemming, makna kata dapat berubah atau hilang, misalnya kata mengeringkan dan kulit kering bisa disamakan menjadi kering, padahal artinya berbeda.
Selain itu, model Word2Vec justru membutuhkan bentuk asli kata agar dapat menangkap konteks penggunaan secara akurat [Mikolov et al. 2013](https://arxiv.org/abs/1301.3781).

---
### B-3.3. Feature Extraction

In [ ]:
# ubah text ke token list
df['tokens'] = df['cleaned_description'].apply(lambda x: x.split())

# latih model Word2Vec
model_w2v = Word2Vec(
    sentences=df['tokens'],
    vector_size=100,  # dimensi embedding
    window=5,
    min_count=2,
    sg=1,  # skip-gram
    workers=4,
    epochs=20
)

Model Word2Vec dilatih menggunakan daftar token (`df['tokens']`) yang berisi kata-kata dari deskripsi produk.
Berikut penjelasan setiap parameter yang digunakan dalam inisialisasi model:

- vector_size=100  
Menentukan jumlah dimensi dari vektor representasi setiap kata.
Nilai 100 dipilih karena cukup untuk menangkap hubungan semantik antar kata tanpa membuat model terlalu besar atau lambat dilatih.

- window=5  
Menentukan ukuran konteks, yaitu jumlah kata di sekitar target kata yang dipertimbangkan saat pelatihan.
Nilai 5 berarti model melihat hingga 5 kata sebelum dan sesudah target kata, cukup ideal untuk teks pendek seperti deskripsi produk.

- min_count=2  
Hanya kata yang muncul minimal 2 kali yang akan dimasukkan ke dalam kosakata model.
Tujuannya untuk mengabaikan kata yang terlalu jarang muncul agar model fokus pada kata yang memiliki konteks kuat dan bermakna umum.

- sg=1  
Mengaktifkan Skip-gram (bukan CBOW).
Skip-gram dipilih karena lebih baik dalam mempelajari representasi untuk kata yang jarang muncul. Ini cocok untuk dataset kecil seperti 788 deskripsi produk.

- workers=4  
Menentukan jumlah core CPU yang digunakan untuk mempercepat proses pelatihan secara paralel.

- epochs=20  
Menentukan berapa kali seluruh data dilatih ulang untuk menyempurnakan bobot model.
Nilai 20 memberikan keseimbangan antara akurasi dan waktu pelatihan, sehingga model dapat stabil memahami hubungan antar kata.

In [ ]:
def get_avg_w2v(tokens, model):
    vectors = [model.wv[w] for w in tokens if w in model.wv]
    if len(vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

# representasi tiap deskripsi sebagai rata-rata vektor kata
df['w2v_vector'] = df['tokens'].apply(lambda x: get_avg_w2v(x, model_w2v))

Fungsi `get_avg_w2v()` digunakan untuk mengubah kumpulan kata dari setiap deskripsi produk menjadi satu vektor angka yang mewakili makna keseluruhan teks.
Fungsi ini akan mengambil vektor Word2Vec dari setiap kata yang dikenali model, lalu menghitung nilai rata-ratanya.
Jika tidak ada kata yang cocok di dalam kosakata model, maka fungsi akan mengembalikan vektor berisi nol dengan ukuran yang sama seperti dimensi model.
Hasil akhirnya disimpan dalam kolom `w2v_vector`, yang berisi representasi numerik dari setiap deskripsi produk.

In [ ]:
len(model_w2v.wv.index_to_key)

3435

In [ ]:
model_w2v.wv['bersih'].shape

(100,)

---
### B-3.4. Similarity Computation

In [ ]:
user_query = "kulit kering dan kusam"
query_tokens = user_query.lower().split()
query_vec = get_avg_w2v(query_tokens, model_w2v).reshape(1, -1)

# ubah semua vektor review ke bentuk 2D
X_vectors = np.vstack(df['w2v_vector'].values)

# cosine similarity
cos_sim = cosine_similarity(query_vec, X_vectors)
top_indices = cos_sim[0].argsort()[-5:][::-1]

df.iloc[top_indices][['product_type', 'product', 'brand', 'description']]

,product_type,product,brand,description
13,cleanser,Face Foam Centella Asiatica,NPURE,Membantu membuat kulit wajah bersih dari sisa ...
520,moisturizer,White Perfect Clinical Overnight Treatment,L'Oreal Paris,Loreal White Perfect Clinical Overnight Treatm...
501,moisturizer,Skin Food Cream,Viva Cosmetics,Viva Cosmetics Skin Food Cream dapat memberika...
96,cleanser,Facial Treatment,Ertos,Meringkas pori wajahMengangkat komedo & kulit ...
615,moisturizer,Skin Food Extra Care,Viva Cosmetics,Viva Cosmetics Skin Food Extra Care adalah kri...


Kode ini digunakan untuk mencari produk yang paling mirip secara makna dengan keluhan pengguna menggunakan pendekatan cosine similarity.
Pertama, teks masukan dari pengguna ("kulit kering dan kusam") diubah menjadi huruf kecil dan dipisahkan menjadi token kata.
Kemudian, setiap kata diubah menjadi vektor Word2Vec dan dirata-ratakan menggunakan fungsi `get_avg_w2v()`, menghasilkan satu vektor representatif untuk query tersebut.
Selanjutnya, semua vektor deskripsi produk diubah menjadi bentuk dua dimensi (`X_vectors`) agar bisa dibandingkan dengan vektor query.
Perhitungan cosine similarity dilakukan untuk mengukur tingkat kemiripan antara keluhan pengguna dan setiap deskripsi produk.
Akhirnya, lima produk dengan nilai kemiripan tertinggi ditampilkan, mencakup jenis produk, nama, merek, dan deskripsinya.

In [ ]:
df.columns

Index(['product_type', 'brand', 'product', 'rating', 'review_count', 'url',
       'image_url', 'price', 'description', 'unique_id', 'cleaned_description',
       'tokens', 'w2v_vector'],
      dtype='object')

In [ ]:
def recommended_products(query, df, model, alpha=0.7, beta=0.15, gamma=0.15, top_n=3):
    # copy df biar tidak merubah data asli
    df_temp = df.copy()

    # Preprocess query
    query_tokens = query.lower().split()
    query_vec = get_avg_w2v(query_tokens, model).reshape(1, -1)
    X_vectors = np.vstack(df_temp['w2v_vector'].values)

    query_clean = preprocess_text(query)
    query_tokens = query_clean.split()

    # Hitung cosine similarity
    cos_sim = cosine_similarity(query_vec, X_vectors)[0]
    df_temp['similarity'] = cos_sim

    # Normalisasi rating
    df_temp['rating_norm'] = (
        (df_temp['rating'] - df_temp['rating'].min()) /
        (df_temp['rating'].max() - df_temp['rating'].min())
    )

    # Normalisasi review count
    df_temp['review_norm'] = (
        (df_temp['review_count'] - df_temp['review_count'].min()) /
        (df_temp['review_count'].max() - df_temp['review_count'].min())
    )

    # Skor gabungan (similarity + rating + review_count)
    df_temp['final_score'] = (
        alpha * df_temp['similarity'] +
        beta * df_temp['rating_norm'] +
        gamma * df_temp['review_norm']
    )

    # Ambil top produk per kategori
    return (
        df_temp.sort_values(by='final_score', ascending=False)
        .groupby('product_type', group_keys=False)
        .head(top_n)
        .sort_values(by='product_type', ascending=True)
        .reset_index(drop=True)
        [['product_type', 'product', 'brand', 'price', 'rating', 'review_count',
          'description', 'similarity', 'final_score']]
    )

Fungsi `recommended_products()` digunakan untuk menghasilkan rekomendasi produk skincare yang paling relevan dengan keluhan pengguna.
Pertama, fungsi menyalin dataset agar data asli tidak berubah, lalu melakukan preprocessing pada teks masukan pengguna dan menghitung vektor representasinya menggunakan model Word2Vec.
Setelah itu, dilakukan perhitungan cosine similarity antara vektor query dan setiap deskripsi produk untuk mengetahui tingkat kemiripan makna.
Selain kemiripan teks, fungsi juga mempertimbangkan rating dan jumlah ulasan (review_count) yang kemudian dinormalisasi agar berada pada skala yang sama.
Ketiga faktor ini digabungkan menggunakan bobot α = 0.7, β = 0.15, dan γ = 0.15 untuk menghasilkan skor akhir (`final_score`).
Terakhir, sistem menampilkan beberapa produk terbaik per kategori dengan nilai skor tertinggi sebagai hasil rekomendasi.

In [ ]:
# Contoh hasil rekomendasi
recommended_products("Kulit kering dan kusam", df, model_w2v)

,product_type,product,brand,price,rating,review_count,description,similarity,final_score
0,cleanser,Gentle Skin Cleanser,Cetaphil,112000.0,4.20,15298,Cetaphil Gentle Skin Cleanser mengandung formu...,0.754621,0.769656
1,cleanser,Face Foam Centella Asiatica,NPURE,90000.0,4.30,1033,Membantu membuat kulit wajah bersih dari sisa ...,0.845682,0.703544
2,cleanser,Bright & Glow Soap,Kojic Plankton,35000.0,4.80,659,Kojic Plankton Bright & Glow Soap merupakan sa...,0.799009,0.701380
3,moisturizer,Water Bank Blue Hyaluronic Cream Oily,Laneige,480800.0,5.00,110,Laneige Water Bank Blue HA Cream OilyKrim pele...,0.794375,0.706513
4,moisturizer,Cerabeads Moisturising Cream,Cosmo Colony,150280.0,4.90,116,CO.CO Cosmo Colony Cerabeads Moisturising Crea...,0.795954,0.700857
5,moisturizer,Hydra Boost Oil Free Moisturizer,ElsheSkin,129000.0,4.70,243,ElsheSkin Hydra Boost Oil-Free Moisturizer mer...,0.793368,0.686628
6,serum/essence,Water Essence,Raiku Beauty,148000.0,4.71,229,Raiku Water Essence bermanfaat untuk memberika...,0.811797,0.700076
7,serum/essence,Radiance Up! Spotless Brightening Serum,YOU Beauty,109000.0,4.80,180,YOU Beauty Radiance Up! Spotless Brightening S...,0.786785,0.688234
8,serum/essence,Radiance Glow Illuminating Serum,YOU Beauty,109000.0,4.80,245,YOU Beauty Radiance Glow Illuminating Serum me...,0.784864,0.687512
9,sun protection,Hydrasoothe Sunscreen Gel SPF 45+++,Azarine Cosmetic,54000.0,4.80,14814,DAILY SKIN PROTECTION FOR SUNSCREEN HATERSORGA...,0.738154,0.794400


Hasil perhitungan similarity dengan query “kulit kering dan kusam” menunjukkan bahwa sistem berhasil menemukan produk-produk yang paling relevan berdasarkan makna deskripsi teksnya.
Produk dengan nilai similarity tertinggi berarti deskripsinya memiliki kemiripan konteks terbesar dengan keluhan tersebut misalnya mengandung kata atau makna yang berkaitan dengan melembapkan, mencerahkan, atau menghidrasi kulit.

Dari tabel, terlihat bahwa beberapa produk seperti Face Foam Centella Asiatica (NPURE) dan Bright & Glow Soap (Kojic Plankton) memiliki nilai similarity di atas 0.8, yang menandakan kemiripan tinggi terhadap permasalahan “kulit kering dan kusam”.
Produk seperti Water Bank Blue Hyaluronic Cream (Laneige) dan Cerabeads Moisturising Cream (Cosmo Colony) juga muncul dengan skor tinggi, karena deskripsinya banyak mengandung istilah yang berhubungan dengan hydration dan moisturizing.

Secara umum, semakin besar nilai cosine similarity, semakin mirip makna deskripsi produk tersebut dengan keluhan pengguna.
Hal ini menunjukkan bahwa model Word2Vec mampu menangkap hubungan semantik antar kata dengan baik, sehingga dapat merekomendasikan produk yang relevan untuk kulit kering dan kusam.

# C. Save Model

In [ ]:
model_w2v.save('word2vec_model.model')

with open('w2v_model.pkl', 'wb') as f:
    pickle.dump(df['w2v_vector'], f)

df.to_pickle('skincare_df_w2v.pkl')

Kode di atas digunakan untuk menyimpan hasil pelatihan model Word2Vec dan data hasil pemrosesan agar bisa digunakan kembali tanpa perlu melatih ulang dari awal.